In [35]:
import openmeteo_requests
import requests_cache
import pandas as pd
from retry_requests import retry

In [166]:
from collections import defaultdict
import pandas as pd
from tqdm.auto import tqdm

In [189]:
cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

openmeteo_params = {
    "latitude": 55.755864,
    "longitude": 37.617698,
    "start_date": "2023-09-01",
    "end_date": "2024-05-30",
    "hourly": ["temperature_2m", "apparent_temperature", "weather_code"]
}
openmeteo_response = openmeteo.weather_api('https://historical-forecast-api.open-meteo.com/v1/forecast', params=openmeteo_params)[0]

hourly = openmeteo_response.Hourly()
hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
hourly_apparent_temperature = hourly.Variables(1).ValuesAsNumpy()
hourly_weather_code = hourly.Variables(2).ValuesAsNumpy()

hourly_data = {"date": pd.date_range(
    start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
    end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
    freq = pd.Timedelta(seconds = hourly.Interval()),
    inclusive = "left"
)}
hourly_data["temperature_2m"] = hourly_temperature_2m
hourly_data["apparent_temperature"] = hourly_apparent_temperature
hourly_data["weather_code"] = hourly_weather_code

hourly_df = pd.DataFrame(data=hourly_data)
hourly_df.date = hourly_df.date.dt.date
hourly_df

,date,temperature_2m,apparent_temperature,weather_code
0,2023-09-01,16.452000,16.591597,0.0
1,2023-09-01,16.052000,16.213207,0.0
2,2023-09-01,15.602000,15.700584,0.0
3,2023-09-01,14.902000,15.089319,2.0
4,2023-09-01,15.352000,15.628864,0.0
...,...,...,...,...
6547,2024-05-30,22.101999,21.284279,3.0
6548,2024-05-30,21.252001,20.501266,3.0
6549,2024-05-30,19.851999,19.441168,2.0
6550,2024-05-30,19.402000,18.975554,2.0


In [192]:
weather_code_mapping = {
    0: 'Ясно',
    1: 'В основном ясно',
    2: 'Переменная облачность',
    3: 'Пасмурно',
    71: 'Cнегопад',
    73: 'Снегопад',
    85: 'Снегопад',
    61: 'Дождь',
    45: 'Туман',
    80: 'Ливень',
    77: 'Град'
}

In [151]:
date = hourly_df.date.unique()[160]
date

datetime.date(2024, 2, 8)

In [153]:
temp_3_days_around = hourly_df[(hourly_df.date - date).isin(pd.timedelta_range('-3 day', periods=7))].temperature_2m
temp_day = hourly_df[hourly_df.date == date].temperature_2m

temp_min_3_days = temp_3_days_around.min()
print(temp_min_3_days)
temp_max_3_days = temp_3_days_around.max()
print(temp_max_3_days)
temp_var_3_days = temp_max_3_days - temp_min_3_days
print(temp_var_3_days)
temp_mean_3_days = temp_3_days_around.mean()
print(temp_mean_3_days)
temp_median_3_days = temp_3_days_around.median()
print(temp_median_3_days)
temp_shift_3_days = temp_3_days_around.iloc[-1] - temp_3_days_around.iloc[0]
print(temp_shift_3_days)

print()

temp_min_day = temp_day.min()
print(temp_min_day)
temp_max_day = temp_day.max()
print(temp_max_day)
temp_var_day = temp_max_day - temp_min_day
print(temp_var_day)
temp_mean_day = temp_day.mean()
print(temp_mean_day)
temp_median_day = temp_day.median()
print(temp_median_day)
temp_shift_day = temp_day.iloc[-1] - temp_day.iloc[0]
print(temp_shift_day)



-18.898
-0.49800003
18.400002
-9.327464
-8.973
-4.85

-14.498
-10.848
3.6500006
-13.062584
-13.273001
-2.4500008


In [164]:
date_to_openmeteo_meta = defaultdict(dict)
for date in tqdm(hourly_df.date.unique()):
    temp_3_days_around = hourly_df[(hourly_df.date - date).isin(pd.timedelta_range('-3 day', periods=7))].temperature_2m
    temp_day = hourly_df[hourly_df.date == date].temperature_2m

    temp_min_3_days = temp_3_days_around.min()
    temp_max_3_days = temp_3_days_around.max()
    temp_var_3_days = temp_max_3_days - temp_min_3_days
    temp_mean_3_days = temp_3_days_around.mean()
    temp_median_3_days = temp_3_days_around.median()
    temp_shift_3_days = temp_3_days_around.iloc[-1] - temp_3_days_around.iloc[0]

    temp_min_day = temp_day.min()
    temp_max_day = temp_day.max()
    temp_var_day = temp_max_day - temp_min_day
    temp_mean_day = temp_day.mean()
    temp_median_day = temp_day.median()
    temp_shift_day = temp_day.iloc[-1] - temp_day.iloc[0]
    
    str_date = str(date)
    date_to_openmeteo_meta[str_date]['temp_min_3_days'] = temp_min_3_days
    date_to_openmeteo_meta[str_date]['temp_max_3_days'] = temp_max_3_days
    date_to_openmeteo_meta[str_date]['temp_var_3_days'] = temp_var_3_days
    date_to_openmeteo_meta[str_date]['temp_mean_3_days'] = temp_mean_3_days
    date_to_openmeteo_meta[str_date]['temp_median_3_days'] = temp_median_3_days
    date_to_openmeteo_meta[str_date]['temp_shift_3_days'] = temp_shift_3_days
    
    date_to_openmeteo_meta[str_date]['temp_min_day'] = temp_min_day
    date_to_openmeteo_meta[str_date]['temp_max_day'] = temp_max_day
    date_to_openmeteo_meta[str_date]['temp_var_day'] = temp_var_day
    date_to_openmeteo_meta[str_date]['temp_mean_day'] = temp_mean_day
    date_to_openmeteo_meta[str_date]['temp_median_day'] = temp_median_day
    date_to_openmeteo_meta[str_date]['temp_shift_day'] = temp_shift_day
    
date_to_openmeteo_meta = dict(date_to_openmeteo_meta)

  0%|          | 0/273 [00:00<?, ?it/s]